# Sentinel-2 Individual Plume Analysis Tool

# Overview
This tool processes Sentinel-2 data to measure methane emissions, particularly from oil and gas fields. It follows this approach:

1. **Data Retrieval**: Downloads Sentinel-2 data for user-defined dates from a pre-coded list of sites.

2. **Preprocessing**: Processes shortwave infrared (SWIR) bands, which are sensitive to methane. Then it compares data from an active emission day with a "no emission" day using a Multi-Band Multi-Pass method.

3. **Plume Analysis**: Identifies areas with elevated methane levels using a thresholding method. Then it measures the total methane in the image, the length of the plume, and downloads wind speed data from the ERA5 API.

4. **Result**: Calculates the emission rate in kg/h using the Integrated Mass Enhancement (IME) method, along with the uncertainty range.

This workflow enables accurate estimation of methane emissions from satellite data and quantifies their impact based on plume characteristics and environmental factors.

The section below imports the packages needed to run the script.


In [ ]:
# Core System and Numerical Operations
import numpy as np  # For numerical operations and array manipulations
import pandas as pd  # For handling tabular data (e.g., CSV files)

# File Handling and Temporary Files
from tempfile import NamedTemporaryFile  # For creating temporary files

# Data Analysis and Manipulation
import xarray as xr  # For working with multidimensional arrays (e.g., NetCDF files)
import cdsapi  # For accessing the Copernicus Climate Data Store API
import requests  # For making HTTP requests (e.g., downloading data)
import openeo  # For cloud-based geospatial data processing

# Geospatial Data Handling
import rasterio  # For working with raster data
from rasterio.plot import show  # For visualising raster data

# Mathematical and Geometric Computations
from scipy.ndimage import label  # For segmentation and labelling of regions

# Interactive Maps and Visualisation
import matplotlib.pyplot as plt  # For plotting and visualisation

# radiative transfer model imports
import setup
import radtran as rt
import time
import scipy
from scipy import optimize
from scipy import interpolate
import scipy.ndimage as ndimage


## Connect to OpenEO

The code below establishes a connection with the Copernicus openEO platform which provides a wide variety of earth observation datasets

- If this does not read as 'Authorised successfully' or 'Authenticated using refresh token', then please ensure that you have completed the setup steps as outlined in section 2.3.6 of the how to guide. 

- If you have followed the steps in section 2.3.6 correctly and the problem persists, please look at https://dataspace.copernicus.eu/news for any information about service interruptions. 

- If there is no news of service problems you can raise a ticket here: https://helpcenter.dataspace.copernicus.eu/hc/en-gb/requests/new

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

## Dispaly field names and select site_id. 

This loads the plume list. The plume boudings need to be manually inputted into an .csv file with the column headdings: 
- id
- name
- west
- south
- east
- north


In [ ]:
studysite_csv = pd.read_csv(r'C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\Individual_Plume_Boundings.csv')
pd.set_option('display.max_rows', None)
print(studysite_csv.to_string(index=False))

## Plume choice

In the code box below, specify the plume number we are interested in for analysis. 

<p style="text-align: center;"><b>site_id</b> = 5</p>

In [ ]:
# Path to the CSV file
csv_file_path = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\Individual_Plume_Boundings.csv"

# Load the dataset
studysite_csv = pd.read_csv(csv_file_path)

site_id = 3  # Specify the oil and gas field ID for the field you want to examine.

# Retrieve the name of the field from the dataset
field_name = studysite_csv[studysite_csv['id'] == site_id].iloc[0]['name']

# Print a confirmation message
print(f"Site {site_id} ({field_name}) loaded correctly.")



# Multi-Band Multi-Pass Analysis

Varon et al. (2021) showed that methane plumes from point sources could be imaged by differencing Sentinel-2’s SWIR-1 and SWIR-2 bands. The tool runs an analysis using a  multi-band-multi-pass retrieval method: 

First it calculates a multi-band-single-pass calculation for both active emission and no emission dates, resulting in two datasets which are then used together for a multi-band-multi-pass method. 
The multi-band-single-pass equation is as follows: 


<div align="center"><b>MBSP = cB12 - B11 / B11  </b></div>

Where:
- <b>B12</b> is the Sentinel-2 SWIR-2 band.
- <b>B11</b> is the Sentinel-2 SWIR-1 band. 
- <b>c</b> is calculated by least-squares fitting B12 to B11 across the scene.  

Once active emission and no emission scenes have been calculated, the following equation is used to calculate the multi-band-multi-pass raster. 

<div align="center"><b>MBMP = ActiveMBSP − NoMBSP</b></div>

Where:
- <b>ActiveMBSP</b> is the multiband single pass for the active emission scene
- <b>NoMBSP</b> is the multiband single pass for the no emission scene.  

The active emission scene and no emission scene are considered in this analysis to be one satelite pass apart. To begin this process we need to determine what days have available satelite data. 

# Available dates for the analysis. 

Sentinel 2 provides data aproximately once every 2 - 3 days, so not every date you can enter into this tool is valid. The code below will tell you what dates are available to use for the oil/gas field of your choice. 

The one parameter you need to modify before running the code is: 

- <b>temporal_extent</b> = ["2020-01-01", "2020-01-31"] (change this to your chosen date range using "YYYY-MM-DD" format.)

Once you have done this run the code and the available dates should appear below in a matter of seconds. 

In [ ]:
def get_spatial_extent(site_id):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]
    return {
        "west": site['west'],
        "south": site['south'],
        "east": site['east'],
        "north": site['north']
    }

def fetch_available_dates(site_id, temporal_extent):
    spatial_extent = get_spatial_extent(site_id)
    catalog_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?box={spatial_extent['west']}%2C{spatial_extent['south']}%2C{spatial_extent['east']}%2C{spatial_extent['north']}&sortParam=startDate&sortOrder=ascending&page=1&maxRecords=1000&status=ONLINE&dataset=ESA-DATASET&productType=L2A&startDate={temporal_extent[0]}T00%3A00%3A00Z&completionDate={temporal_extent[1]}T00%3A00%3A00Z&cloudCover=%5B0%2C{cloud_cover}%5D"
    response = requests.get(catalog_url)
    response.raise_for_status()
    catalog = response.json()
    dates = [date.split('T')[0] for date in map(lambda x: x['properties']['startDate'], catalog['features'])]
    return dates

# Please enter your perameters here.
temporal_extent = ["2019-10-01", "2019-11-30"]  # Specify the the date range you want to check for available data.
cloud_cover = 5

available_dates = fetch_available_dates(site_id, temporal_extent)
print("Available dates:", available_dates)

## Choosing the "Active Emission" Date

A so called active emission date must be chosen from one of the available datasets. This will be the chosen day we are looking for plumes.  

Like before, the one parameter you need to modify before running the code is:

<p style="text-align: center;"><b>temporal_extent</b> = ["2020-01-17", "2020-01-17"]</p>

Change this to your chosen date range using "YYYY-MM-DD" format. 

Please note that the temporal extent dates <b><u>MUST BE IDENTICAL</u></b> because we are only choosing a single date.

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.

In [ ]:
def active_emission(site_id, active_temporal_extent):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]

    active_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=active_temporal_extent,
        spatial_extent={
            "west": site['west'],
            "south": site['south'],
            "east": site['east'],
            "north": site['north']
        },
        bands=["B11", "B12", "sunZenithAngles", "viewZenithMean"],
    )
    active_emission.download("Sentinel-2_active_emissionMBMP.Tiff")
    
    sun_zenith_mean = None
    view_zenith_mean = None

    with rasterio.open("Sentinel-2_active_emissionMBMP.Tiff") as src:
        band_names = ["B11", "B12", "sunZenithAngles", "viewZenithMean"]
        means = {band_names[i]: np.mean(src.read(i+1)) for i in range(src.count)}

    print(f"Mean value of B11: {means['B11']}")
    print(f"Mean value of B12: {means['B12']}")
    print(f"Mean value of sunZenithAngles: {means['sunZenithAngles']}")
    print(f"Mean value of viewZenithMean: {means['viewZenithMean']}")

    return means["sunZenithAngles"], means["viewZenithMean"]

# Enter parameters for the active emission day
active_temporal_extent = ["2019-11-20", "2019-11-20"]

# Store results for later use
sun_zenith_mean, view_zenith_mean = active_emission(site_id, active_temporal_extent)



## Choosing the "No Emission" Date

Next we choose the no emission date using the same process. This is the dataset we will compare the "Active Emission" one too. The recommended choice is the satelite overpass immediately before the "Active Emission" one. 

<b>So if your active emission day is 2020-01-17, your no emission day would be 2020-01-14</b>

In an ideal world, the "No Emission" day should contain no emissions, but in fields with a lot of activity like Hassi Messaoud, this may not be possible. Such an instance will not cause problems in most cases. The emissions for these dates will simply appear as dark clouds on the SWIR data and can be ignored in the analysis. 

The one parameter you need to modify before running the code is:

<p style="text-align: center;"><b>temporal_extent</b> = ["2020-01-14", "2020-01-14"]</p>

The temporal extent dates <b><u>MUST BE IDENTICAL</u></b>

If you recieve an error message of 'NoDataAvailable' then please check the list of available data above and try again.


In [ ]:
def no_emission(site_id, temporal_extent):
    site = studysite_csv[studysite_csv['id'] == site_id].iloc[0]

    no_emission = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=no_temporal_extent,
        spatial_extent={
            "west": site['west'],
            "south": site['south'],
            "east": site['east'],
            "north": site['north']
        },
        bands=["B11", "B12"],
    )
    no_emission.download("Sentinel-2_no_emissionMBMP.Tiff")

# Enter perameters for the active emission day
no_temporal_extent = ["2019-10-06", "2019-10-06"]

no_emission(site_id, no_temporal_extent)

## Running Plume Analysis

The following code box performs the MBMP analysis as outlined earlier. Not mentioned earlier is that the sentinel-2 data is divided by 10,000 to obtain the reflectance values as per the documentation on Units in: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S2L1C.html

In [ ]:
def least_squares_fit_no_intercept(x, y):
    """Computes least-squares scaling factor c (forcing intercept = 0)."""
    mask = ~np.isnan(x) & ~np.isnan(y)
    x_valid = x[mask]
    y_valid = y[mask]
    
    # Compute scaling factor c as the least-squares solution to y = c * x
    c = np.sum(x_valid * y_valid) / np.sum(x_valid ** 2)  # Least squares slope with zero intercept
    
    return c

# Define file paths
Active_Multiband = "Sentinel-2_active_emissionMBMP.Tiff"
No_Multiband = "Sentinel-2_no_emissionMBMP.Tiff"
output_file = "SWIR_diff.tiff"

# Open datasets and perform least squares fitting
with rasterio.open(Active_Multiband) as Active_img, rasterio.open(No_Multiband) as No_img:
    # Read data and convert to float for safe division
    Active_B11 = Active_img.read(1).astype(float) / 10000.0
    Active_B12 = Active_img.read(2).astype(float) / 10000.0
    No_B11 = No_img.read(1).astype(float) / 10000.0
    No_B12 = No_img.read(2).astype(float) / 10000.0

    # Compute scaling factor c for each pass (forcing intercept to 0)
    c_active = least_squares_fit_no_intercept(Active_B11.flatten(), Active_B12.flatten())
    c_no = least_squares_fit_no_intercept(No_B11.flatten(), No_B12.flatten())

    # Correct Band 12 using computed c values
    Corrected_Active_B12 = c_active * Active_B12
    Corrected_No_B12 = c_no * No_B12

    # Compute MBSP retrieval using the correct methodology equation
    MBSP_active = c_active * (Active_B12 - Active_B11) / Active_B11
    MBSP_no = c_no * (No_B12 - No_B11) / No_B11

    # Compute MBMP difference (Final MBMP retrieval)
    SWIR_diff = MBSP_active - MBSP_no


# Save SWIR_diff
with rasterio.open(Active_Multiband) as src:
    meta = src.meta.copy()
    meta.update({
        "count": 1,
        "dtype": SWIR_diff.dtype
    })
    with rasterio.open(output_file, "w", **meta) as dest:
        dest.write(SWIR_diff, 1)

# Print output file name and its values as a numpy array
print(f"Output file saved as: {output_file}")

import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Define output file path
output_file = "SWIR_diff.tiff"

# Open the output file and read data
with rasterio.open(output_file) as src:
    SWIR_diff_data = src.read(1)  # Read the first (and only) band
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]  # Get spatial extent

# Mask NaN or NoData values
SWIR_diff_data = np.ma.masked_where(SWIR_diff_data == src.nodata, SWIR_diff_data)

# Plot the data
plt.figure(figsize=(10, 6))
plt.imshow(SWIR_diff_data, cmap='coolwarm', extent=extent, origin="upper")
plt.colorbar(label="SWIR Difference Value")
plt.title("SWIR Difference Map")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()




## Radiative Transfer Model Part 1 - Functions

The following section is part 1 of the radiative transfer model referred to in Varon et al. 2021, kindly given by Dr Daniel Varon. The first code box defines a series of functions used by the model. 

In [ ]:
# Copyright (C) 2020 GHGSat Inc.

'''
THIS LICENSE AGREEMENT (“LICENSE”) IS A LEGAL AGREEMENT BETWEEN YOU (“USER”) AND GHGSAT INC. (“GHGSAT”), LOCATED AT 500-3981 ST-LAURENT, MONTREAL, QC, CANADA H2W1Y5 GOVERNING THE GHGSAT SENTINEL-2 METHANE SOFTWARE CODE (THE “SOFTWARE”). USER WILL BE DEEMED TO HAVE ACCEPTED AND AGREED TO THE TERMS AND CONDITIONS OF THIS LICENSE IF USER DOWNLOADS AND/OR USES THE SOFTWARE. 
1.	OWNERSHIP: The Software is protected by copyright law and is also confidential information; it is licensed for limited purposes.  All title in and to the Software and all intellectual property rights in or related thereto, including any copy, translation, modification, or adaptation of the Software will remain the exclusive property of GHGSAT INC. (“GHGSAT”).  
2.	GRANT OF LICENSE:  GHGSAT grants to User a limited, non-transferable, non-exclusive, perpetual license for academic research and non-commercial use (the “Internal Use”) to utilise the Software and any accompanying written materials, and anything derived therefrom, solely as set forth in this License (the “Grant of License”).  
3.	PERMITTED USES:  User agrees and understands that it MAY: a. make an unlimited number of copies of the Software for Internal Use only; b. provide the Software to collaborators directly related to Internal Use of the Software all of whom must agree (i) to maintain confidentiality of the Software under terms no less restrictive than User’s duty hereunder and (ii) that they will not retain the Software or copies thereof after completion of User’s Internal Use; c. store, post or process the Software in a system that is not accessible by the public, and commensurate with standards regarding the protection of sensitive data; and d. publish research incorporating the Software provided that User first notifies GHGSat of its intent to publish such research and gives GHGSat adequate opportunity to ensure the Software is accurately represented. User shall not alter, cover, remove or otherwise interfere with any copyright notice(s) inscribed on/in the Software. Any approved publication or other work that incorporates the Software must conspicuously acknowledge the following: “GHGSAT Data and Products – Copyright © 2021 GHGSAT Inc. All rights reserved.”
4.	WARRANTY:  GHGSAT is supplying the Software “as is”. GHGSAT GIVES NO WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT
5.	LIMITATION ON LIABILITY: IN NO EVENT WILL GHGSAT BE LIABLE FOR DAMAGES RELATING TO THE SOFTWARE OR OTHERWISE ARISING OUT OF, RELATED TO, OR IN ANY WAY CONNECTED WITH THIS LICENSE, REGARDLESS OF THE FORM OF ACTION, WHETHER BASED ON CONTRACT, NEGLIGENCE, PRODUCT LIABILITY, TRADE PRACTICES, OR OTHERWISE, INCLUDING CONSEQUENTIAL, INDIRECT, SPECIAL, PUNITIVE, OR INCIDENTAL DAMAGES OR LOST PROFITS, WHETHER FORESEEABLE OR UNFORESEEABLE, OF ANY KIND. THE LIMITATIONS CONTAINED IN THIS SECTION ARE NOT MADE WHERE PROHIBITED BY LAW. 
6.	MISCELLANEOUS: (1) This License will be terminated as soon as User fails to comply with or is in breach of these terms and User shall promptly destroy the Software or return it to GHGSat. (2) The laws of the Province of Ontario, Canada govern this License. 
7.	CONFIDENTIALITY: The User acknowledges that the Software and the data contained therein are valuable intellectual property and that part of the value therein derives from maintaining confidentiality of the Software and the associated data. Accordingly, the User agrees that it shall hold the Software and the associated data in the strictest confidence and will not disclose same to anyone other than to those of its collaborators who need to have access to the Software and its associated data for the purposes of Internal Use. 
'''

# Global variables
c = 299792458.
kB = 1.38064852e-23
deg = np.pi/180
mbartoatm = 0.000986923
kpatoatm = 0.00986923/1000.
kmtocm = 1E5
mtocm = 100.
mol=6.022140857E23
start_time = time.time()

# Key directories
aux_data_dir = 'aux_data'
hapi_data_dir = 'hapi_data'

def importMRTP(num_layers, targheight, obsheight, solarangle, obsangle, ch4_scale=1.1029, co2_scale=1.2424, h2o_scale=1):
    '''
    Load auxiliary data for vertical profiles of temperature, pressure, and CH4, CO2, and H2O mixing ratio.
    These data come from the U.S. Standard Atmosphere. 
    Also calculate the pathlength in each layer for input observation angles.
    CH4 and CO2 profiles need to be scaled up to match modern-day concentrations.

    Arguments
        num_layers [int]   : Number of vertical pressure layers for the radiative transfer calculation
        targheight [float] : Target elevation above sea level in km
        obsheight  [float] : Altitude of satellite instrument in km. Can use 100 km as if satellite were at top of atmosphere
        solarangle [float] : Solar zenith angle in degrees
        obsangle   [float] : Viewing zenith angle of instrument in degrees
        ch4_scale  [float] : Scale factor multiplying U.S. Standard CH4 profile (1.1029 implies 1875 ppb at the surface)
        co2_scale  [float] : Same but for U.S. Standard CO2 profile (1.2424 implies 410 ppm at the surface)
        h2o_scale  [float] : Same but for U.S. Standard H2O profile

    Returns
        pathlength [float] : Path length in each layer [cm]
        pressavg   [float] : Average pressure in each layer [atm]
        tempavg    [float] : Average temperature in each layer [K]
        mrCH4avg   [float] : Average CH4 mixing ratio in each layer
        mrCO2avg   [float] : Average CO2 mixing ratio in each layer
        mrH2Oavg   [float] : Average H2O mixing ratio in each layer
    ''' 
        
    # Mixing Ratio
    mrCH4 = np.transpose(np.genfromtxt(f'{aux_data_dir}/ch4.dat'))
    mrCO2 = np.transpose(np.genfromtxt(f'{aux_data_dir}/co2.dat'))
    mrH2O = np.transpose(np.genfromtxt(f'{aux_data_dir}/h2o.dat'))
    
    # Temperature
    temp = np.transpose(np.genfromtxt(f'{aux_data_dir}/temperature.dat'))
    
    # Pressure
    press = np.transpose(np.genfromtxt(f'{aux_data_dir}/pressure.dat'))
    
    # Define altitude in cm
    altitude = press[1][::-1] * kmtocm
    # Define pressure in atm
    pressure = press[0][::-1] * mbartoatm
    
    # Find pressure as a function of altitude
    altfine = np.arange(0, 10, 0.01)
    pressfine = np.interp(altfine, press[1], press[0]*mbartoatm)
    idfine = (np.abs(altfine-targheight)).argmin()
    pressmax = pressfine[idfine]    
    
    # Interpolate to get layers evenly spaced in pressure
    dpress = (pressmax-2.9608E-5)/num_layers
    pressinterp = np.arange(0, pressmax+dpress, dpress)
    altinterp = np.interp(pressinterp, pressure, altitude)

    # Find temperature at each altitude
    alttemp = temp[1]*kmtocm
    temperature = temp[0]
    tempinterp = np.interp(altinterp, alttemp, temperature)
    
    # Find mixing ratios at each altitude
    altmrCH4 = mrCH4[1]*kmtocm
    altmrCO2 = mrCO2[1]*kmtocm
    altmrH2O = mrH2O[1]*kmtocm
    mixrateCH4 = mrCH4[0]*ch4_scale
    mixrateCO2 = mrCO2[0]*co2_scale
    mixrateH2O = mrH2O[0]*h2o_scale
    
    # Interpolate and then sample using the altitude sample points determined from isobaric pressure increases
    mrCH4interp = np.interp(altinterp, altmrCH4, mixrateCH4)
    mrCO2interp = np.interp(altinterp, altmrCO2, mixrateCO2)
    mrH2Ointerp = np.interp(altinterp, altmrH2O, mixrateH2O)
    
    def find_nearest_alt(array,value):
        idx = (np.abs(array-value)).argmin()
        secondpass = array[idx:len(array)]
        zeroarray = np.zeros(idx)
        upwellingpass = np.concatenate((zeroarray, secondpass))
        return upwellingpass
    
    upwellingpass = find_nearest_alt(altinterp, obsheight*kmtocm)
    
    # Find path length of each layer
    pathlengthdown = np.zeros(num_layers)
    pathlengthup = np.zeros(num_layers)
    for i in range(0, num_layers):
        pathlengthdown[i] = np.absolute(altinterp[i]-altinterp[i+1])
        pathlengthup[i] = np.absolute(upwellingpass[i]-upwellingpass[i+1])
        
    # Calculate path given the Solar and observation angle from Nadir
    pathlength = pathlengthdown/np.cos(solarangle*deg) + pathlengthup/np.cos(obsangle*deg)
 
    # Define average value in layers
    pressavg = np.zeros(len(pathlength))
    tempavg = np.zeros(len(pathlength))
    mrCH4avg = np.zeros(len(pathlength))
    mrCO2avg = np.zeros(len(pathlength))
    mrH2Oavg = np.zeros(len(pathlength))   
    for i in range(0,len(pathlength)):
        pressavg[i] = (pressinterp[i+1]+pressinterp[i])/2.
        tempavg[i] = (tempinterp[i+1]+tempinterp[i])/2.
        mrCH4avg[i] = (mrCH4interp[i+1]+mrCH4interp[i])/2.
        mrCO2avg[i] = (mrCO2interp[i+1]+mrCO2interp[i])/2.
        mrH2Oavg[i] = (mrH2Ointerp[i+1]+mrH2Ointerp[i])/2.

    return pathlength, pressavg, tempavg, mrCH4avg, mrCO2avg, mrH2Oavg

def radtran(targheight, obsheight, solarangle, obsangle, instrument, band, num_layers=100):
    '''
    Computes the top-of-atmosphere spectral radiance (TOASR) for an input instrument and spectral band.

    Arguments
        targheight [float] : Target elevation above sea level in km
        obsheight  [float] : Altitude of satellite instrument in km. Can use 100 km as if satellite were at top of atmosphere
        solarangle [float] : Solar zenith angle in degrees
        obsangle   [float] : Viewing zenith angle of instrument in degrees
        instrument [str]   : MSI instrument. Choose 'S2A' or 'S2B'
        band       [int]   : Spectral band. Choose 11 or 12
        num_layers [int]   : Number of vertical pressure layers for the radiative transfer calculation

    Returns
        toasr          [float] : Band-integrated top-of-atmosphere spectral radiance [W/m2/m/sr]
        odCH4pts       [float] : CH4 optical depth by wavelength
        odCO2pts       [float] : CO2 optical depth by wavelength
        odH2Opts       [float] : H2O optical depth by wavelength
        solar_spectrum [float] : Upwelling solar spectrum
        cdCH4          [float] : CH4 slant column density in mol/m2
    '''

    start_time = time.time()   
    
    print('Creating the transmission spectrum...')
    
    # Import pressure, temperature, path-length, and mixing ratios
    (L_cm, press_atm, temp, mrCH4, mrCO2, mrH2O) = importMRTP(num_layers, targheight, obsheight, solarangle, obsangle)
    
    # Load absorption cross_sections        
    wavelength = np.load(f'{hapi_data_dir}/abs_wave_hapi_{instrument}_band{band}.npy')
    press_load = np.load(f'{hapi_data_dir}/abs_press_hapi_{instrument}_band{band}.npy')
    temp_load = np.load(f'{hapi_data_dir}/abs_temp_hapi_{instrument}_band{band}.npy')   
    absCH4_load  = np.load(f'{hapi_data_dir}/abs_ch4_hapi_{instrument}_band{band}.npy')
    absCO2_load = np.load(f'{hapi_data_dir}/abs_co2_hapi_{instrument}_band{band}.npy')
    absH2O_load = np.load(f'{hapi_data_dir}/abs_h2o_hapi_{instrument}_band{band}.npy')
    num_wave = len(wavelength)

    # Get solar spectrum
    solarspec = np.transpose(np.genfromtxt(f'{aux_data_dir}/SUNp01_4000_to_7000.txt'))
    wavesolar = 1E7/solarspec[0][::-1]
    radiancesolar = solarspec[1][::-1]*(100*solarspec[0][::-1]**2)
    solarradiance = np.interp(wavelength,wavesolar,radiancesolar)
    
    # Calculate optical density
    odCH4pts_upper = np.zeros(num_wave)
    odCH4pts_lower = np.zeros(num_wave)
    odCO2pts = np.zeros(num_wave)
    odH2Opts = np.zeros(num_wave)
    
    interp_order = 3
    for i in range(num_wave):
        
        fCH4_tp = scipy.interpolate.RectBivariateSpline(temp_load, press_load, absCH4_load.T[i], kx=interp_order, ky=interp_order)
        fCO2_tp = scipy.interpolate.RectBivariateSpline(temp_load, press_load, absCO2_load.T[i], kx=interp_order, ky=interp_order)
        fH2O_tp = scipy.interpolate.RectBivariateSpline(temp_load, press_load, absH2O_load.T[i], kx=interp_order, ky=interp_order)
               
        for j in range(num_layers):
            
            # Calculate density
            temperature_K = temp[j]
            pressure_atm = press_atm[j]
            pressure_Pa = pressure_atm*101325
            density_m3 = pressure_Pa/(kB*temperature_K)
            density_cm3 = density_m3/(1E6)
            
            # Evaluate interpolation function
            f_CH4_temp = fCH4_tp(temperature_K, pressure_atm)
            f_CO2_temp = fCO2_tp(temperature_K, pressure_atm)
            f_H2O_temp = fH2O_tp(temperature_K, pressure_atm)
                        
            # Calculate the (unit-less) optical density: OD = abs*n*MR*L
            lim_low = 6
            if j >= num_layers - lim_low:
                # Lowest 6 pressure layers = lowest 500 m of atmosphere (lower)
                odCH4pts_lower[i] = odCH4pts_lower[i] + f_CH4_temp*density_cm3*mrCH4[j]*L_cm[j]
            else:
                # The rest of the atmosphere (upper)
                odCH4pts_upper[i] = odCH4pts_upper[i] + f_CH4_temp*density_cm3*mrCH4[j]*L_cm[j]
            odCO2pts[i] = odCO2pts[i] + f_CO2_temp*density_cm3*mrCO2[j]*L_cm[j]
            odH2Opts[i] = odH2Opts[i] + f_H2O_temp*density_cm3*mrH2O[j]*L_cm[j]

    # Calculate slant column density of methane
    cdCH4 = np.sum((press_atm/kpatoatm/(kB*temp)/mtocm**3)*mrCH4*L_cm/mol*mtocm**2)

    press_atm_lower = press_atm[-lim_low:]
    temp_lower = temp[-lim_low:]
    L_cm_lower = L_cm[-lim_low:]
    cdCH4_lower = np.sum((press_atm_lower/kpatoatm/(kB*temp_lower)/mtocm**3)*mrCH4[-lim_low:]*L_cm_lower/mol*mtocm**2)

    # Calculate the Top-Of-Atmosphere Spectral Radiance (TOASR) in the band [W/m2/m/sr]
    solar_spectrum = solarradiance/np.pi * np.cos(solarangle*deg)
    toasr = np.mean(np.exp(-(odCH4pts_lower + odCH4pts_upper + odCO2pts + odH2Opts)) * solar_spectrum)
    
    print("--- %s seconds --- to run radtran()" % (time.time() - start_time))

    return toasr, odCH4pts_lower, odCH4pts_upper, odCO2pts, odH2Opts, solar_spectrum, cdCH4, cdCH4_lower

def retrieve(frac_refl_data, instrument, method, targheight, obsheight, solarangle, obsangle, num_layers=100):
    '''
    Infer methane column enhancements from fractional reflectance measurements.

    Arguments
        frac_refl_data [float] : Array of fractional reflectance data, deltaR = (cR-R0)/R0
                                 e.g., DeltaR_SBMP from eq. (1) in Varon et al. 2021 AMT
        instrument     [str]   : MSI instrument. Choose 'S2A' or 'S2B'
        method         [str]   : Retrieval method corresponding to frac_refl_data
                                 Choose 'MBSP' or 'SBMP'
        targheight     [float] : Target elevation above sea level in km
        obsheight      [float] : Altitude of satellite instrument in km. Can use 100 km as if satellite were at top of atmosphere
        solarangle     [float] : Solar zenith angle in degrees
        obsangle       [float] : Viewing zenith angle of instrument in degrees
        num_layers     [int]   : Number of vertical pressure layers for the radiative transfer calculation
    '''

    # Choose method
    if method == 'SBMP':
        
        # Get toasr, optical depths, etc. from radtran()
        toasr_12, odCH4_lower_12, odCH4_upper_12, odCO2_12, odH2O_12, solar_spectrum_12, cdCH4, cdCH4_lower = radtran(targheight, obsheight, solarangle, obsangle, instrument, band=12, num_layers=100)

        def frac_abs_SBMP_difference(ch4_enh, data): 
            '''
            Fractional absorption model to compare with measurements for SBMP method.
            
            Arguments
                ch4_enh [float] : Modeled enhancement as fraction of background
                data    [float] : Actual (cR-R0)/R0
            '''
                
            ch4 = ch4_enh + 1
            toasr_CH4enh_12 = np.mean(np.exp(-(ch4*odCH4_lower_12 + odCH4_upper_12 + odCO2_12 + odH2O_12))*solar_spectrum_12)
            frac_abs_SBMP = (toasr_CH4enh_12 - toasr_12)/toasr_12
            
            return frac_abs_SBMP - data
    
    elif method == 'MBSP':

        # Get toasr, optical depths, etc. from radtran()
        toasr_11, odCH4_lower_11, odCH4_upper_11, odCO2_11, odH2O_11, solar_spectrum_11, _, _ = radtran(targheight, obsheight, solarangle, obsangle, instrument, band=11, num_layers=num_layers)
        toasr_12, odCH4_lower_12, odCH4_upper_12, odCO2_12, odH2O_12, solar_spectrum_12, cdCH4, cdCH4_lower = radtran(targheight, obsheight, solarangle, obsangle, instrument, band=12, num_layers=num_layers)
        
        def frac_abs_MBSP_difference(ch4_enh, data):
            '''
            Fractional absorption model to compare with measurements for MBSP method.
            
            Arguments
                ch4_enh [float] : Modeled enhancement as fraction of background
                data    [float] : Actual (cR-R0)/R0
            '''
                
            ch4 = ch4_enh + 1
            toasr_CH4enh_12 = np.mean(np.exp(-(ch4*odCH4_lower_12 + odCH4_upper_12 + odCO2_12 + odH2O_12))*solar_spectrum_12)
            toasr_CH4enh_11 = np.mean(np.exp(-(ch4*odCH4_lower_11 + odCH4_upper_11 + odCO2_11 + odH2O_11))*solar_spectrum_11)

            frac_abs_12 = (toasr_CH4enh_12 - toasr_12)/toasr_12
            frac_abs_11 = (toasr_CH4enh_11 - toasr_11)/toasr_11
            
            frac_abs_MBSP = frac_abs_12 - frac_abs_11 
                
            return frac_abs_MBSP - data

    else:
        raise ValueError('Bad method selection. Must be "MBSP" or "SBMP".')
     
    start_time = time.time()

    # Do retrieval
    (num_rows, num_cols) = frac_refl_data.shape
    ch4_out = np.zeros((num_rows, num_cols))

    for i in range(num_rows):
        for j in range(num_cols):

            data_temp = frac_refl_data[i,j]
                
            if np.isnan(data_temp):
                print('Found nan, skipping.')
                ch4_out[i,j] = np.nan
            
            # Solve for the best-fit fractional column (fraction of background)
            else:
                if method == 'SBMP':
                    ch4_temp = scipy.optimize.newton(lambda ch4_scale: frac_abs_SBMP_difference(ch4_scale,data_temp), 0, rtol=0.0001, maxiter=10000, disp=False)
                elif method == 'MBSP':
                    ch4_temp = scipy.optimize.newton(lambda ch4_scale: frac_abs_MBSP_difference(ch4_scale,data_temp), 0, rtol=0.0001, maxiter=10000, disp=False)

                # Convert the fractional column to absolute vertical column density in mol/m2   
                AMF = 1/np.cos(obsangle*deg) + 1/np.cos(solarangle*deg)
                ch4_out[i,j] = ch4_temp * (cdCH4_lower/cdCH4)*(cdCH4/AMF)
        
    # Time    
    print("--- %s seconds --- to optimize" % (time.time() - start_time))

    return ch4_out

## Radiative Transfer Model Part 2 - Download HAPI data 

If this is the fist time you're runnning this tool you will need to download the requisite HAPI data. If so, please set this to <b>"True"</b>. If you have already done this, it can be set to <b>"False"</b> so that this legnthy process is avoided. 

In [ ]:
# Set to True if you need to setup your data directory
run_setup = False

if run_setup:
    setup.setup_retrieval_directory('S2A','hapi_data')

## Radiative Transfer Model Part 3 - Reflectance Values to mol/m2

This section converts the reflectance data of Sentinel-2 to usable scientific units (mol/m2). 

The following section may reqire some user input. The target height will need to be changed for the area of study to reflect its altitude. 

In [ ]:
# Configuration
num_layers = 100
targheight = 0.17 # in kilometres 
obsheight = 786 # in kilometres
solarangle = sun_zenith_mean
obsangle = view_zenith_mean
instrument = 'S2A' # Sentinel-2A or 2B
method = 'SBMP' # We use SBMP in the case of MBMP as they process the same. 

# Path to your .tiff file
tiff_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\SWIR_diff.tiff"

# Read the .tiff file
with rasterio.open(tiff_file) as src:
    # Read the image data into a numpy array
    frac_refl_data = src.read(1)  # Assumes the data is in the first band

# Try the column retrieval
methane_column_retrieval = rt.retrieve(frac_refl_data, instrument, method, targheight, obsheight, solarangle, obsangle, num_layers=num_layers)

methane_column_retrieval # units mol/m2

# Save the output array as a new .tiff file
output_tiff = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\CH4_Concentration.tiff"

# Get the metadata of the original .tiff file to use the same attributes
with rasterio.open(tiff_file) as src:
    metadata = src.meta

# Update metadata for the output file, ensure it's 1-band (grayscale)
metadata.update(dtype=rasterio.float32, count=1)

# Write the new data to a .tiff file
with rasterio.open(output_tiff, 'w', **metadata) as dst:
    dst.write(methane_column_retrieval.astype(rasterio.float32), 1)  # Ensure the data is written as float32

print(f"Output saved to: {output_tiff}")


## Plume Masking

Now the radiative transfer model has completed its work, we need to identify the plume's extent. This code box uses a 95 percentile thresholding method to determine which pixels constitute the plume. 

In [ ]:
# File paths
input_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\CH4_Concentration.tiff"
output_mask_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\plume_mask.tiff"

# Load the methane enhancement raster
with rasterio.open(input_file) as src:
    methane_data = src.read(1).astype(float)  # Read the first band
    metadata = src.meta.copy()  # Copy metadata for output file

# Step 1: Compute the 95th percentile threshold
percentile_95 = np.nanpercentile(methane_data, 95)

# Step 2: Create a binary plume mask
plume_mask = np.where(methane_data >= percentile_95, 1, 0)

# Step 3: Apply a 3×3 median filter for noise reduction
plume_mask_filtered = ndimage.median_filter(plume_mask, size=3)

# Step 4: Save the plume mask as a new TIFF file
metadata.update(dtype=rasterio.uint8, count=1, nodata=None)  # Ensure nodata is removed

with rasterio.open(output_mask_file, "w", **metadata) as dst:
    dst.write(plume_mask_filtered.astype(rasterio.uint8), 1)

# Print confirmation and threshold value
print(f"Plume mask saved to: {output_mask_file}")
print(f"95th percentile threshold for methane enhancement: {percentile_95}")




## Plume Dimentions

This code box measures the length and area of the plume base on the mask. <b>The length currently doesn't work and so you will need to measure this in a GIS.</b>

In [ ]:
# File path for plume mask
plume_mask_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\plume_mask.tiff"

# Load the plume mask
with rasterio.open(plume_mask_file) as src:
    plume_mask = src.read(1).astype(float)  # Read binary plume mask (1 = plume, 0 = background)
    transform = src.transform  # Get spatial transformation info

# Step 1: Compute pixel area in square meters
pixel_width = transform.a  # X-resolution (meters per pixel)
pixel_height = -transform.e  # Y-resolution (meters per pixel, negative because of image orientation)
pixel_area = pixel_width * pixel_height  # Area of a single pixel in square meters

# Step 2: Compute total plume area (number of plume pixels * pixel area)
plume_area = np.sum(plume_mask) * pixel_area  # Sum all plume pixels (value=1) and multiply by pixel area

# Step 3: Compute plume length scale L
L = np.sqrt(plume_area)

# Print results
print(f"Pixel size: {pixel_width} x {pixel_height} meters")
print(f"Total plume area: {plume_area:.2f} m²")
print(f"Computed plume length scale (L): {L:.2f} m")


## Wind Speed

The next code box detemines wind speed using the ERA5 API which is then adjusted calibration parameters from large-eddy simulations that model how wind affects methane plumes from Varon er al. (2021).

<div align="center"><b>U_eff = 0.33 * U_10 + 0.45</b></div>

Where:
- <b>U_eff</b> is the effective wind speed.
- <b>U_10</b> is tthe local 10m wind speed.  

There will be a long warning message but it can be ignored. 

In [ ]:
csv_path = r'C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Data\Algerian_Oil_and_Gas_Fields.csv'


# Function to extract bounding box and calculate center from site_id
def get_location_from_site_id(site_id, csv_path):
    """
    Extract center latitude and longitude for a site based on site_id.

    Args:
    - site_id (int): The ID of the site to extract.
    - csv_path (str): Path to the CSV containing site boundaries.

    Returns:
    - dict: Dictionary with latitude and longitude of the center.
    """
    df = pd.read_csv(csv_path)
    site = df[df['id'] == site_id]
    if site.empty:
        raise ValueError(f"Site ID {site_id} not found in the CSV file.")
    site = site.iloc[0]
    center_lat = (site['south'] + site['north']) / 2
    center_lon = (site['west'] + site['east']) / 2
    return {'latitude': center_lat, 'longitude': center_lon}

# Get the location for the ERA5 data request
location = get_location_from_site_id(site_id, csv_path)

# Initialize the CDS API client
c = cdsapi.Client()

# Define parameters for the data request
# Extract the start date from active_temporal_extent and assign it to date
date = active_temporal_extent[0]  # Use the first element as the single date

# Now the variable 'date' can be used with the other API

# date = active_temporal_extent (update this somehow so that no input is needed in this box)

# Retrieve ERA5 data and store it in a temporary file
with NamedTemporaryFile(suffix='.nc') as tmp_file:
    result = c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind'],
            'year': date.split('-')[0],
            'month': date.split('-')[1],
            'day': date.split('-')[2],
            'time': ['10:00'],  # Specify time of interest
            'format': 'netcdf',  # NetCDF format
            'area': [
                location['latitude'] + 0.25, location['longitude'] - 0.25,
                location['latitude'] - 0.25, location['longitude'] + 0.25,
            ],  # Small bounding box around the location
        }
    )
    # Download data to the temporary file
    result.download(tmp_file.name)
    
    # Load the dataset with xarray
    ds = xr.open_dataset(tmp_file.name)

# Extract u and v components
u10 = ds['u10'].sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest')
v10 = ds['v10'].sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest')

# Calculate wind speed
wind_speed = np.sqrt(u10**2 + v10**2)

# Handle single timestep case
if 'time' in u10.dims:
    # Multiple timesteps (not likely in this case since we specified 10:00 only)
    for time, speed in zip(u10.time.values, wind_speed.values):
        print(f"{time}: Wind Speed = {speed:.2f} m/s")
else:
    # Single timestep
    wind_speed_value = wind_speed.values.item()  # Convert array to scalar
    

# Plume wind adjustments Constants from Varon et al. 2021
alpha = 0.33
beta = 0.45  # in m/s

# Compute the effective wind speed
U_eff = alpha * wind_speed_value + beta

# Print result
print(f"10m Wind Speed (U10) from ERA5: {wind_speed_value:.2f} m/s")
print(f"Effective Wind Speed (Ueff): {U_eff:.2f} m/s")


## Calculating total methane in kg

The next codebox calculates the total amount of methane in the masked area in kg. 

In [ ]:
# Define file paths
methane_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\CH4_Concentration.tiff"
plume_mask_file = r"C:\GIS_Course\Methane_Point_Detection\Sentinel-2_Algeria_Methane\Varon methane detection code\s2_ch4_code_share\plume_mask.tiff"

# Constants
M_CH4 = 0.016  # kg/mol (molar mass of methane)

# Load methane enhancement data
with rasterio.open(methane_file) as src:
    methane_data = src.read(1).astype(float)  # Read methane enhancement (mol/m²)
    transform = src.transform  # Get pixel size information

# Load plume mask
with rasterio.open(plume_mask_file) as src:
    plume_mask = src.read(1).astype(bool)  # Read plume mask (1 = plume, 0 = background)

# Compute pixel area (same method as before)
pixel_width = transform.a  # X-resolution (m per pixel)
pixel_height = -transform.e  # Y-resolution (m per pixel, negative because of orientation)
pixel_area = pixel_width * pixel_height  # m² per pixel

# Compute IME: sum of (methane enhancement * pixel area * molar mass), only for plume pixels
IME = np.sum(methane_data[plume_mask] * pixel_area * M_CH4)

# Print result
print(f"Computed Integrated Methane Enhancement (IME): {IME:.4f} kg")


## Emission rate calculation

The next section calculates the emission rate based on the total mass of methane in kg (IME), the effective windspeed (U_eff) and the plume legnth (L). Results are given in kg per second and per hour.  

In [ ]:
# Constants
# IME = 2554.6573  # kg 
# U_eff = 1.06  # m/s 
# L = 358  # Placeholder: Replace with actual plume length scale from Step 2

# Compute source rate Q (kg/s)
Q_kg_s = (IME * U_eff) / L

# Convert to kg/h
Q_kg_h = Q_kg_s * 3600

# Print result
print(f"Estimated Methane Source Rate: {Q_kg_s:.4f} kg/s")
print(f"Estimated Methane Source Rate: {Q_kg_h:.2f} kg/h")



## Uncertainty 

The final code box calculates the +- uncertanty values for the emission rate. 

In [ ]:
U_10 = wind_speed_value

# Use the median RMSE from the study for inland terrain
wind_error = 1.34  # Median RMSE value for ERA5 wind speed over inland terrain
# source: https://www.sciencedirect.com/science/article/pii/S1364032122006293

# Compute relative wind uncertainty
wind_error_percent = wind_error / U_10  
sigma_wind = wind_error_percent * Q_kg_h  # Wind uncertainty contribution

# Uncertainty estimates from study
retrieval_error = 0.13  # Column retrieval precision (mol/m², from study)
IME_error_percent = 0.15  # IME model error (15% of IME)
multi_pass_error_percent = 0.01  # Multi-pass uncertainty (1%)

# Compute individual uncertainties
sigma_retrieval = retrieval_error * Q_kg_h  # Retrieval error contribution
sigma_IME = IME_error_percent * Q_kg_h  # IME model error contribution
sigma_multi_pass = multi_pass_error_percent * Q_kg_h  # Multi-pass retrieval error contribution

# Compute total uncertainty using quadrature addition
sigma_total = np.sqrt(sigma_wind**2 + sigma_retrieval**2 + sigma_IME**2 + sigma_multi_pass**2)

# Compute uncertainty bounds
Q_lower = Q_kg_h - sigma_total
Q_upper = Q_kg_h + sigma_total

# Print results
print(f"Estimated Methane Source Rate: {Q_kg_h:.2f} kg/h")
print(f"Total Uncertainty (1σ): ±{sigma_total:.2f} kg/h")
print(f"Final Source Rate Range: {Q_lower:.2f} to {Q_upper:.2f} kg/h")

